### 라이브러리

In [200]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib 
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['font.family'] = "AppleGothic"

from sklearn.ensemble import RandomForestRegressor

from pycaret.regression import *
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error

### 데이터 처리

In [222]:
train = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/train.csv')
test = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')
submission = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/sample_submission.csv')

In [223]:
weekdayrankfordinner = {
    '월' : 1,
    '화' : 2,
    '수' : 4,
    '목' : 3,
    '금' : 5
} 

train['요일(석식)'] = train['요일'].map(weekdayrankfordinner)
test['요일(석식)'] = test['요일'].map(weekdayrankfordinner)

In [224]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day
train['주'] = train['일자'].dt.isocalendar().week
train['요일'] = train['일자'].dt.weekday


test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day
test['주'] = test['일자'].dt.isocalendar().week
test['요일'] = test['일자'].dt.weekday

train['주'] = train['주'].astype('int') # '주'의 타입이 Uint로 사용이 안되어 int로 변경해줌.
test['주'] = test['주'].astype('int')

In [225]:
drops = ['조식메뉴', '중식메뉴', '석식메뉴']

train = train.drop(drops, axis=1)
test = test.drop(drops, axis=1)

In [226]:
train['식사가능자수'] = (train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수'])
test['식사가능자수'] = (test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수'])

In [227]:
month1 = {
    1:3,
    2:1,
    3:2,
    4:6,
    5:7,
    6:8,
    7:10,
    8:9,
    9:5,
    10:4,
    11:11,
    12:12
}

month2 = {
    1:11,
    2:2,
    3:1,
    4:4,
    5:8,
    6:6,
    7:9,
    8:7,
    9:5,
    10:3,
    11:10,
    12:12
}
train['월(중식)'] = train['월'].map(month1)
test['월(중식)'] = test['월'].map(month1)

train['월(석식)'] = train['월'].map(month2)
test['월(석식)'] = test['월'].map(month2)

train['월(중식)'] = train['월(중식)'].astype('int') # float형이라 int로 변경
test['월(중식)'] = test['월(중식)'].astype('int')
train['월(석식)'] = train['월(석식)'].astype('int')
test['월(석식)'] = test['월(석식)'].astype('int')

In [228]:
train['공휴일전후'] = 0
test['공휴일전후'] = 0

In [229]:
train['공휴일전후'][4] = 1 #2
train['공휴일전후'][17] = 1 #
#train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후'][152] = 1
train['공휴일전후'][226] = 1
train['공휴일전후'][221] = 1
#train['공휴일전후'][222] = 1
train['공휴일전후'][224] = 1
train['공휴일전후'][225] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후'][310] = 2
train['공휴일전후'][311] = 1
train['공휴일전후'][309] = 1
train['공휴일전후'][330] = 1
train['공휴일전후'][379] = 1
train['공휴일전후'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후'][470] = 1
train['공휴일전후'][502] = 2
# train['공휴일전후'][501] = 1
train['공휴일전후'][511] = 1
train['공휴일전후'][565] = 1
train['공휴일전후'][623] = 1
train['공휴일전후'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후'][705] = 2
# train['공휴일전후'][707] = 1
train['공휴일전후'][709] = 2
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후'][815] = 2
train['공휴일전후'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후'][950] = 1
train['공휴일전후'][951] = 1
train['공휴일전후'][953] = 2
train['공휴일전후'][954] = 1
train['공휴일전후'][955] = 1
train['공휴일전후'][971] = 1
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후'][1038] = 1
train['공휴일전후'][1099] = 1
train['공휴일전후'][1129] = 2
# train['공휴일전후'][1128] = 1
train['공휴일전후'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후'][10] =2
test['공휴일전후'][20] = 1

In [230]:
# 원핫인코딩
train = pd.get_dummies(train, columns=['공휴일전후'])
test = pd.get_dummies(test, columns=['공휴일전후'])


In [231]:
test['공휴일전후_0'][20] = 1
test['공휴일전후_1'][20] = 0

In [232]:
train[train.석식계>0]['석식계'].mean() # 0이아닌값 평균

# 현재 1월만 대체
train['석식계'][244] = 478.8605851979346

#train['석식계'][281] = 398
train['석식계'][492] = 478.8605851979346

#train['석식계'][502] = 398
#train['석식계'][510] = 398
#train['석식계'][529] = 398
train['석식계'][730] = 478.8605851979346

#train['석식계'][747] = 398
#train['석식계'][766] = 398
train['석식계'][973] = 478.8605851979346

#train['석식계'][993] = 398

# 1 2 3 월 자기개발의날 대체.

### 최종제출 ( 자기개발1대체,금)

In [233]:
train.loc[(train.요일==4), '석식계'] -= 10

In [234]:
train.loc[(train.요일==0), '중식계'] -= 50
train.loc[(train.요일==1), '중식계'] -= 70
train.loc[(train.요일==2), '중식계'] -= 60
train.loc[(train.요일==3), '중식계'] -= 70

In [235]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [236]:
# train = train[train.석식계!= 0]

x1_train = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월(중식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']] #중식계
x2_train = train[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수','월(석식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']] #석식계

y1_train = train['중식계']
y2_train = train['석식계']

x1_test = test[['요일', '본사시간외근무명령서승인건수', '식사가능자수', '월(중식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']]
x2_test = test[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수', '월(석식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']]

In [237]:
model1 = cat
model2 = cat

model1.fit(x1_train, y1_train) #중식계
pred1 = model1.predict(x1_test)

model2.fit(x2_train, y2_train) #석식계
pred2 = model2.predict(x2_test)

submission['중식계'] = pred1
submission['석식계'] = pred2

Learning rate set to 0.040347
0:	learn: 194.8407755	total: 6.03ms	remaining: 6.02s
1:	learn: 190.3387524	total: 8.13ms	remaining: 4.05s
2:	learn: 185.7825159	total: 10.3ms	remaining: 3.41s
3:	learn: 181.5089848	total: 12.2ms	remaining: 3.04s
4:	learn: 177.7007597	total: 14.3ms	remaining: 2.85s
5:	learn: 173.9928170	total: 16.2ms	remaining: 2.68s
6:	learn: 170.5355409	total: 18ms	remaining: 2.56s
7:	learn: 166.8795903	total: 19.9ms	remaining: 2.46s
8:	learn: 163.4257347	total: 21.8ms	remaining: 2.4s
9:	learn: 160.4796589	total: 24.2ms	remaining: 2.39s
10:	learn: 157.3678949	total: 25.9ms	remaining: 2.33s
11:	learn: 154.3158589	total: 29.9ms	remaining: 2.46s
12:	learn: 151.4298972	total: 32.4ms	remaining: 2.46s
13:	learn: 148.8575862	total: 34.5ms	remaining: 2.43s
14:	learn: 146.6506142	total: 36.3ms	remaining: 2.39s
15:	learn: 144.4181723	total: 51.8ms	remaining: 3.19s
16:	learn: 142.2197638	total: 53.6ms	remaining: 3.1s
17:	learn: 140.1389782	total: 55.2ms	remaining: 3.01s
18:	learn: 1

In [238]:
submission.head(3)

,일자,중식계,석식계
0,2021-01-27,972.949671,321.318684
1,2021-01-28,892.223096,507.473362
2,2021-01-29,569.451354,233.021222


In [161]:
submission

,일자,중식계,석식계
0,2021-01-27,990.608270,321.318684
1,2021-01-28,922.131649,507.473362
2,2021-01-29,571.231622,233.021222
3,2021-02-01,1203.204051,518.737171
4,2021-02-02,1003.901046,569.793711
5,2021-02-03,1026.067827,515.775621
6,2021-02-04,951.073494,565.792804
7,2021-02-05,742.490950,369.800979
8,2021-02-08,1253.301074,683.592530
9,2021-02-09,1033.707604,585.701251


In [117]:
submission['석식계'].mean()

524.463584091015

In [241]:
submission.to_csv('자기개발1대체,금다운중식플마.csv', index=False)

In [36]:
train.groupby('요일').mean()['중식계']

요일
0    1146.395833
1     925.620833
2     905.213389
3     823.991803
4     653.590909
Name: 중식계, dtype: float64

In [41]:
train.query(' 년==2020 ').groupby('요일').mean()['중식계']

요일
0    1136.043478
1     936.000000
2     889.319149
3     813.229167
4     634.200000
Name: 중식계, dtype: float64

In [92]:
submission.query('석식계>200')

,일자,중식계,석식계
0,2021-01-27,1034.788569,321.318684
1,2021-01-28,980.601533,507.473362
2,2021-01-29,580.084097,233.021222
3,2021-02-01,1226.842164,518.737171
4,2021-02-02,1049.531797,569.793711
5,2021-02-03,1064.153606,515.775621
6,2021-02-04,991.497225,565.792804
7,2021-02-05,733.553544,369.800979
8,2021-02-08,1297.061915,683.592530
9,2021-02-09,1083.387400,585.701251


In [239]:
submission['일자'] = pd.to_datetime(submission['일자'])
submission['요일'] = submission['일자'].dt.weekday

In [240]:
submission.groupby('요일').mean()['중식계']

요일
0    1208.146275
1    1014.170186
2     960.299879
3     883.574468
4     662.344028
Name: 중식계, dtype: float64

In [220]:
submission.mean()

중식계    907.333068
석식계    524.463584
요일       2.040000
dtype: float64

In [221]:
train.query('월 <=4 & 월 >=1 & 년 == 2020').groupby('요일').mean()['중식계']

요일
0    1073.062500
1     856.882353
2     814.687500
3     754.705882
4     681.250000
Name: 중식계, dtype: float64